In [ ]:
! pip install trulens_eval==0.15.3 langchain>=0.0.263 unstructured>=0.0.1 chromadb>=0.0.1

In [ ]:
import os
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate


from langchain.chat_models import ChatOpenAI
from langchain.chains import  RetrievalQA

from langchain import OpenAI

from langchain.agents import Tool
from langchain.agents import initialize_agent
from langchain.memory import ConversationSummaryBufferMemory
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

import openai


from trulens_eval import TruChain, Feedback, Tru
from trulens_eval.feedback import OpenAI as fOpenAI

os.environ["OPENAI_API_KEY"] = "..."
tru = Tru()


### Define custom class that loads dcouments into local vector store (Chroma instance)

In [ ]:

class VectorstoreManager:
    def __init__(self):
        self.vectorstore = None  # Vectorstore for the current conversation
        self.all_document_splits = []  # List to hold all document splits added during a conversation

    def initialize_vectorstore(self):
        """Initialize an empty vectorstore for the current conversation."""
        self.vectorstore = Chroma(
            embedding_function=OpenAIEmbeddings(), 
        )
        self.all_document_splits = []  # Reset the documents list for the new conversation
        return self.vectorstore

    def add_documents_to_vectorstore(self, url_lst: list):
        """Example assumes loading new documents from websites to the vectorstore during a conversation."""
        for doc_url in url_lst:
            document_splits = self.load_and_split_document(doc_url)
            self.all_document_splits.extend(document_splits)
        
        # Create a new Chroma instance with all the documents
        self.vectorstore = Chroma.from_documents(
            documents=self.all_document_splits, 
            embedding=OpenAIEmbeddings(), 
        )

        return self.vectorstore

    def get_vectorstore(self):
        """Provide the initialized vectorstore for the current conversation. If not initialized, do it first."""
        if self.vectorstore is None:
            raise ValueError("Vectorstore is not initialized. Please initialize it first.")
        return self.vectorstore

    @staticmethod
    def load_and_split_document(url: str, chunk_size=1000, chunk_overlap=0):  
        """Load and split a document into chunks."""
        loader = WebBaseLoader(url)
        splits = loader.load_and_split(RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap))
        return splits

In [ ]:

DOC_URL = "http://paulgraham.com/worked.html"

vectorstore_manager = VectorstoreManager()
vec_store = vectorstore_manager.add_documents_to_vectorstore([DOC_URL])

In [ ]:
vectorstore_manager.all_document_splits

### Set up multiple agents for document retrieval and summarization

In [ ]:
llm = ChatOpenAI(
        model_name='gpt-3.5-turbo-16k',
        temperature=0.0    
    )


conversational_memory = ConversationSummaryBufferMemory(
    k=4,
    max_token_limit=64,
    llm=llm,
    memory_key = "chat_history",
    return_messages=True
)


retrieval_summarization_template = """
System: Follow these instructions below in all your responses:
System: always try to retrieve documents as knowledge base or external data source from retriever (vector DB). 
System: If performing summarization, you will try to be as accurate and informational as possible.
System: If providing a summary/key takeaways/highlights, make sure the output is numbered as bullet points.
If you don't understand the source document or cannot find sufficient relevant context, be sure to ask me for more context information.
{context}
Question: {question}
Action:
"""
question_generation_template = """
System: Based on the summarized context, you are expected to generate a specified number of multiple choice questions from the context to ensure understanding. Each question, unless specified otherwise, is expected to have 4 options and only correct answer.
System: Questions should be in the format of numbered list.
{context}
Question: {question}
Action:
"""

answer_generation_template = """
System: You are going to give the correct answers to these 10 multiple choice questions from the {context}, 1 single correct answer for each question, with detailed explanation. 
System: If you are referring back to the multiple choice questions, do not rephrase the question. Just retrieve and present the question verbatim. c
System: Answers to questions should be in the format of numbered list.
{context}
Question: {question}
Action:
"""

summarization_prompt = PromptTemplate(template=retrieval_summarization_template, input_variables=["question", "context"])
answer_generator_prompt = PromptTemplate(template=answer_generation_template, input_variables=["question", "context"])
question_generator_prompt = PromptTemplate(template=question_generation_template, input_variables=["question", "context"])



# retrieval qa chain
summarization_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=vec_store.as_retriever(),
        chain_type_kwargs={'prompt': summarization_prompt}
)

question_chain = RetrievalQA.from_chain_type(llm=llm,
        chain_type="stuff",
        retriever=vec_store.as_retriever(),
        chain_type_kwargs={'prompt': question_generator_prompt}
    )

answer_chain = RetrievalQA.from_chain_type(llm=llm,
        chain_type="stuff",
        retriever=vec_store.as_retriever(),
        chain_type_kwargs={'prompt': answer_generator_prompt}
    )
tools = [
        Tool(
            name="Knowledge Base / retrieval from documents",
            func=summarization_chain.run,
            description="useful for when you need to answer questions about the source document(s).",
        ),
    
        Tool(
            name="Conversational agent to generate multiple choice questions about the summary of the source document(s)",
            func=question_chain.run,
            description="useful for when you need to have a conversation with a human and hold the memory of the current / previous conversation.",
        ),

        Tool(
            name="Conversational agent to give answers about the summary of the source document(s)",
            func=answer_chain.run,
            description="useful for when you need to have a conversation with a human and hold the memory of the current / previous conversation.",
        )
    ]
agent = initialize_agent(
        agent='chat-conversational-react-description',
        tools=tools,
        llm=llm,
        memory=conversational_memory
    )


In [ ]:
(agent)

In [ ]:
type(tools[0]).__bases__

In [ ]:
agent("Please summarize the document into 5 to 10 highlight points")

### Set up Evaluation

In [ ]:
class OpenAI_custom(fOpenAI):
    def no_answer_feedback(self, question: str, response: str) -> float:
        return float(openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
            {"role": "system", "content": "Does the RESPONSE provide an answer to the QUESTION? Rate on a scale of 1 to 10. Respond with the number only."},
            {"role": "user", "content": f"QUESTION: {question}; RESPONSE: {response}"}
        ]
    )["choices"][0]["message"]["content"]) / 10

custom = OpenAI_custom()

# No answer feedback (custom)
f_no_answer = Feedback(custom.no_answer_feedback).on_input_output()

In [ ]:
tru_agent = TruChain(agent, app_id = "Conversational_Retrieval_Agent", feedbacks = [f_no_answer])

In [ ]:
user_prompts = [
    "Please summarize the document to a short summary under 100 words"
]

with tru_agent as recording:
    for prompt in user_prompts:
        agent(prompt)

In [ ]:
%debug